In [18]:
from pyspark.mllib.stat import Statistics

from pyspark.sql import Row
from pyspark.sql.types import *


# Functions

# Load test data

In [62]:
# Load data
wwine_rdd = sc.textFile("wine-data/winequality-white.csv")
wwine_rdd = wwine_rdd.map(lambda line: line.replace('"',''))
wwine_rdd = wwine_rdd.map(lambda line: line.replace(';',','))

In [75]:
# Get header
header_str = wwine_rdd.first()
header = header_str.split(',')
header = map(lambda h: h.replace(' ','_'),header)
Wine = Row(*header)


In [92]:
# Create colection of Rows
data = wwine_rdd.filter(lambda x: x!=header_str)
data = data.map(lambda w: w.split(','))
data = data.map(lambda w: map(float,w))
#data = data.map(lambda w: Wine(*w))

# Create DataFrame
df = sqlContext.createDataFrame(data, schema=header)
df.printSchema()

root
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)



# Univariate selection

In [95]:
# First compute correlation
df.stat.corr('fixed_acidity','volatile_acidity')

-0.02269729014664496

In [94]:
df.select('fixed_acidity').

[Row(fixed_acidity=7.0),
 Row(fixed_acidity=6.3),
 Row(fixed_acidity=8.1),
 Row(fixed_acidity=7.2)]